<a href="https://colab.research.google.com/github/maxmatical/fast.ai/blob/master/fastai_%2B_blurr_%2B_deberta_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install --upgrade fastai
!pip install transformers
!pip install ohmeow-blurr
!pip install bentoml

In [2]:
from fastai.test_utils import show_install
show_install()



```text
=== Software === 
python        : 3.7.10
fastai        : 2.3.1
fastcore      : 1.3.20
fastprogress  : 0.2.7
torch         : 1.8.1+cu101
nvidia driver : 460.32
torch cuda    : 10.1 / is available
torch cudnn   : 7603 / is enabled

=== Hardware === 
nvidia gpus   : 1
torch devices : 1
  - gpu0      : Tesla T4

=== Environment === 
platform      : Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic
distro        : #1 SMP Sat Jun 5 09:50:34 PDT 2021
conda env     : Unknown
python        : /usr/bin/python3
sys.path      : 
/content
/env/python
/usr/lib/python37.zip
/usr/lib/python3.7
/usr/lib/python3.7/lib-dynload
/usr/local/lib/python3.7/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.7/dist-packages/IPython/extensions
/root/.ipython
```

Please make sure to include opening/closing ``` when you paste into forums/github to make the reports appear formatted as code sections.

Optional package(s) to enhance the diagnostics can be installed with:
pip install distro
Onc

In [7]:
from transformers import *
from fastai.text.all import *

from blurr.data.all import *
from blurr.modeling.all import *
import torch.nn as nn

In [4]:
path = untar_data(URLs.IMDB_SAMPLE)

model_path = Path('models')
imdb_df = pd.read_csv(path/'texts.csv')

In [9]:
# updated with ohmeow-blurr==0.0.26
model_cls = AutoModelForSequenceClassification

pretrained_model_name = 'microsoft/deberta-base'
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, model_cls=model_cls)

In [11]:
# single input
blocks = (HF_TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model), CategoryBlock)
dblock = DataBlock(blocks=blocks,  get_x=ColReader('text'), get_y=ColReader('label'), splitter=ColSplitter())

dls = dblock.dataloaders(imdb_df, bs=4)

In [12]:
dls.show_batch(dataloaders=dls, max_n=2)

,text,category
0,"Raising Victor Vargas: A Review<br /><br />You know, Raising Victor Vargas is like sticking your hands into a big, steaming bowl of oatmeal. It's warm and gooey, but you're not sure if it feels right. Try as I might, no matter how warm and gooey Raising Victor Vargas became I was always aware that something didn't quite feel right. Victor Vargas suffers from a certain overconfidence on the director's part. Apparently, the director thought that the ethnic backdrop of a Latino family on the lower east side, and an idyllic storyline would make the film critic proof. He was right, but it didn't fool me. Raising Victor Vargas is the story about a seventeen-year old boy called, you guessed it, Victor Vargas (Victor Rasuk) who lives his teenage years chasing more skirt than the Rolling Stones could do in all the years they've toured. The movie starts off in `Ugly Fat' Donna's bedroom where Victor is sure to seduce her, but a cry from outside disrupts his plans when his best-friend Harold (Kevin Rivera) comes-a-looking for him. Caught in the attempt by Harold and his sister, Victor Vargas runs off for damage control. Yet even with the embarrassing implication that he's been boffing the homeliest girl in the neighborhood, nothing dissuades young Victor from going off on the hunt for more fresh meat. On a hot, New York City day they make way to the local public swimming pool where Victor's eyes catch a glimpse of the lovely young nymph Judy (Judy Marte), who's not just pretty, but a strong and independent too. The relationship that develops between Victor and Judy becomes the focus of the film. The story also focuses on Victor's family that is comprised of his grandmother or abuelita (Altagracia Guzman), his brother Nino (also played by real life brother to Victor, Silvestre Rasuk) and his sister Vicky (Krystal Rodriguez). The action follows Victor between scenes with Judy and scenes with his family. Victor tries to cope with being an oversexed pimp-daddy, his feelings for Judy and his grandmother's conservative Catholic upbringing.<br /><br />The problems that arise from Raising Victor Vargas are a few, but glaring errors. Throughout the film you get to know certain characters like Vicky, Nino, Grandma,",negative
1,"THE SHOP AROUND THE CORNER is one of the sweetest and most feel-good romantic comedies ever made. There's just no getting around that, and it's hard to actually put one's feeling for this film into words. It's not one of those films that tries too hard, nor does it come up with the oddest possible scenarios to get the two protagonists together in the end. In fact, all its charm is innate, contained within the characters and the setting and the plot... which is highly believable to boot. It's easy to think that such a love story, as beautiful as any other ever told, *could* happen to you... a feeling you don't often get from other romantic comedies, however sweet and heart-warming they may be. <br /><br />Alfred Kralik (James Stewart) and Clara Novak (Margaret Sullavan) don't have the most auspicious of first meetings when she arrives in the shop (Matuschek & Co.) he's been working in for the past nine years, asking for a job. They clash from the very beginning, mostly over a cigarette box that plays music when it's opened--he thinks it's a ludicrous idea; she makes one big sell of it and gets hired. Their bickering takes them through the next six months, even as they both (unconsciously, of course!) fall in love with each other when they share their souls and minds in letters passed through PO Box 237. This would be a pretty thin plotline to base an entire film on, except that THE SHOP AROUND THE CORNER is expertly fleshed-out with a brilliant supporting cast made up of entirely engaging characters, from the fatherly but lonely Hugo Matuschek (Frank Morgan) himself, who learns that his shop really is his home; Pirovitch (Felix Bressart), Kralik's sidekick and friend who always skitters out of the room when

In [13]:
model = HF_BaseModelWrapper(hf_model)

learn = Learner(dls, 
                model,
                loss_func=LabelSmoothingCrossEntropyFlat(),
                metrics=[accuracy],
                cbs=[HF_BaseModelCallback],
                splitter=hf_splitter).to_fp16()

learn.unfreeze()

cbs = [
    GradientAccumulation(n_acc=32),
    ReduceLROnPlateau(monitor='accuracy', factor=5, patience=5),
    EarlyStoppingCallback(monitor='accuracy', min_delta=0.01, patience=7),
    SaveModelCallback(monitor="accuracy", fname="model_1")
]



In [14]:
learn.fit_one_cycle(
    3,
    lr_max=3e-5,
    cbs = cbs
)

epoch,train_loss,valid_loss,accuracy,time
0,0.562785,0.511825,0.815000,00:47
1,0.324467,0.337559,0.925000,00:48
2,0.278647,0.353841,0.930000,00:49


Better model found at epoch 0 with accuracy value: 0.8149999976158142.
Better model found at epoch 1 with accuracy value: 0.925000011920929.
Better model found at epoch 2 with accuracy value: 0.9300000071525574.


# Inference and deployment with BentoML

Notess:
- can just treat as a fastai model and use `test_dl` and `get_preds` to make inference
- Can also quantize the underpying pytorch model with `learn.model.hf_model`

## Using fastai

In [15]:
learn.validate()

(#2) [0.3538411259651184,0.9300000071525574]

In [16]:
test_dl = learn.dls.test_dl(["this movie sucks", "i really like this movie"])
learn.get_preds(dl=test_dl, with_decoded=True)

(tensor([[0.9429, 0.0571],
         [0.0548, 0.9452]]), None, tensor([0, 1]))

In [17]:
preds = learn.get_preds(dl=test_dl, with_decoded=True)[-1].cpu().numpy()
preds

array([0, 1])

## Using only the underlying pytorch model (faster)

allows for `torchscript` with jit 

In [ ]:
# save model from learner as .pt
torch.save(learn.model.hf_model, "deberta.pt")
# load model to cpu and change to eval mode
inf_model = torch.load("deberta.pt", map_location=torch.device("cpu"))
inf_model.eval()

### single prediction

In [19]:
text = "This movie sucks"
text_ids = hf_tokenizer.encode(text)
inp = tensor(text_ids)[None]

inf_model(inp)

SequenceClassifierOutput([('logits',
                           tensor([[ 1.5908, -1.3197]], grad_fn=<AddmmBackward>))])

### batch prediction

In [20]:
# tokenizze batch of text
texts = ["This movie sucks", "i really like this movie"]
text_ids = hf_tokenizer.batch_encode_plus(texts, 
                                          padding='longest',
                                          truncation=True,
                                          max_length=256,
                                          return_tensors="pt")

text_ids

{'input_ids': tensor([[    1,   713,  1569, 29384,     2,     0,     0],
        [    1,   118,   269,   101,    42,  1569,     2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1]])}

In [21]:
# creating pytorch dataloader
from torch.utils.data import TensorDataset, BatchSampler
input_ids_inf = text_ids['input_ids']
attention_masks_inf = text_ids['attention_mask']

dataset_inf = TensorDataset(input_ids_inf, attention_masks_inf)
dataloader_inf = DataLoader(dataset_inf, sampler=SequentialSampler(dataset_inf), batch_size=32)

In [22]:
# run batch inference
device = "cpu"
all_logits = np.empty([0,2])
# shape is (0, 2), 2 cols for 2 probabilities (positive and negative)

for batch in dataloader_inf:

    batch = tuple(b.to(device) for b in batch)
    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              }
    with torch.no_grad(): # with torch >= 1.9 can use `with torch.inference_mode():` to be even faster         
        outputs = inf_model(**inputs)

    logits = outputs[0]

    all_logits = np.vstack([all_logits, torch.softmax(logits, dim=1).detach().cpu().numpy()])

In [23]:
all_logits

array([[0.94836175, 0.05163824],
       [0.05483662, 0.94516343]])

Now we can deploy with BentoML using either fastai or pytorch frameworks

## additonal: using jit tracing (faster than pure pytorch)

In [ ]:
# load model to cpu and change to eval mode
inf_model = torch.load("deberta.pt", map_location=torch.device("cpu"))
inf_model.eval()

In [24]:
sample_ids, sample_mask = text_ids["input_ids"][0].numpy(), text_ids["attention_mask"][0].numpy()


In [25]:
sample_ids, sample_mask = tensor([sample_ids]), tensor([sample_mask])
sample_ids, sample_mask 

(tensor([[    1,   713,  1569, 29384,     2,     0,     0]]),
 tensor([[1, 1, 1, 1, 1, 0, 0]]))

In [27]:
# traced_model = torch.jit.trace(inf_model, [sample_ids, sample_mask])
traced_model = torch.jit.trace(inf_model, [sample_ids, sample_mask], strict=False) # using strict=False (because transformer return dict)



/usr/local/lib/python3.7/dist-packages/transformers/models/deberta/modeling_deberta.py:596: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  scale = math.sqrt(query_layer.size(-1) * scale_factor)
/usr/local/lib/python3.7/dist-packages/transformers/models/deberta/modeling_deberta.py:636: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  att_span = min(max(query_layer.size(-2), key_layer.size(-2)), self.max_relative_positions)
/usr/local/lib/python3.7/dist-packages/transformers/models/deberta/modeling_deberta.py:659: TracerWarning: Converting a tensor to a Python

In [ ]:
# save and load jit model
torch.jit.save(traced_model, "traced_model.pt")
# note: deberta uses XSoftmax which is a python function which causes issues for saving

traced_model = torch.jit.load("traced_model.pt")

batch encoding again

In [31]:
# batch encoding, same as before
texts = ["This movie sucks", "i really like this movie"]
text_ids = hf_tokenizer.batch_encode_plus(texts, 
                                          padding='longest',
                                          truncation=True,
                                          max_length=256,
                                          return_tensors="pt")

text_ids

from torch.utils.data import TensorDataset, BatchSampler
input_ids_inf = text_ids['input_ids']
attention_masks_inf = text_ids['attention_mask']

dataset_inf = TensorDataset(input_ids_inf, attention_masks_inf)
dataloader_inf = DataLoader(dataset_inf, sampler=SequentialSampler(dataset_inf), batch_size=32)

In [32]:
# batch inference
device = "cpu"
all_logits = np.empty([0,2])
# shape is (0, 2), 2 cols for 2 probabilities (positive and negative)

for batch in dataloader_inf:

    batch = tuple(b.to(device) for b in batch)
    input_ids, input_mask = batch[0], batch[1]

    with torch.no_grad():        
        outputs = traced_model(input_ids, input_mask)
    logits = outputs["logits"]
    all_logits = np.vstack([all_logits, torch.softmax(logits, dim=1).detach().cpu().numpy()])

all_logits

array([[0.94836175, 0.05163824],
       [0.05483662, 0.94516343]])

## fastest: quantized + jit tracing

In [38]:
def print_model_size(mdl):
    torch.save(mdl.state_dict(), "tmp.pt")
    print("%.2f MB" %(os.path.getsize("tmp.pt")/1e6))
    os.remove('tmp.pt')

In [52]:
# load model to cpu and change to eval mode
inf_model = torch.load("deberta.pt", map_location=torch.device("cpu"))
inf_model.eval()

print_model_size(inf_model)


556.86 MB


In [53]:
# dynamic quantization
model_dynamic_quantized = torch.quantization.quantize_dynamic(
    inf_model, qconfig_spec={torch.nn.Linear}, dtype=torch.qint8
)

print_model_size(model_dynamic_quantized)


257.88 MB


In [64]:
# save quantized model
torch.save(model_dynamic_quantized, "quantized_model.pt")
model_dynamic_quantized = torch.load("quantized_model.pt", map_location=torch.device("cpu"))
print("%.2f MB" %(os.path.getsize("quantized_model.pt")/1e6)) 

257.84 MB


In [54]:
# static quantization
# inf_model.fuse_model()

# backend = "fbgemm" # use qnnpack for arm architecture, usually mobile devices, fbgemm is for x86 architecture
# inf_model.qconfig = torch.quantization.get_default_qconfig(backend)
# torch.backends.quantized.engine = backend
# model_static_quantized = torch.quantization.prepare(inf_model, inplace=False)
# model_static_quantized = torch.quantization.convert(model_static_quantized, inplace=False) # currenty not working due to embedding issues

# print_model_size(model_static_quantized)

In [55]:
# jit trace model again
sample_ids, sample_mask = text_ids["input_ids"][0].numpy(), text_ids["attention_mask"][0].numpy()
sample_ids, sample_mask = tensor([sample_ids]), tensor([sample_mask])
traced_q_model = torch.jit.trace(model_dynamic_quantized, [sample_ids, sample_mask], strict=False) # using strict=False (because transformer return dict)


/usr/local/lib/python3.7/dist-packages/transformers/models/deberta/modeling_deberta.py:596: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  scale = math.sqrt(query_layer.size(-1) * scale_factor)
/usr/local/lib/python3.7/dist-packages/transformers/models/deberta/modeling_deberta.py:636: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  att_span = min(max(query_layer.size(-2), key_layer.size(-2)), self.max_relative_positions)
/usr/local/lib/python3.7/dist-packages/transformers/models/deberta/modeling_deberta.py:659: TracerWarning: Converting a tensor to a Python

run inference

In [65]:
# batch encoding, same as before
texts = ["This movie sucks", "i really like this movie"]
text_ids = hf_tokenizer.batch_encode_plus(texts, 
                                          padding='longest',
                                          truncation=True,
                                          max_length=256,
                                          return_tensors="pt")

text_ids

from torch.utils.data import TensorDataset, BatchSampler
input_ids_inf = text_ids['input_ids']
attention_masks_inf = text_ids['attention_mask']

dataset_inf = TensorDataset(input_ids_inf, attention_masks_inf)
dataloader_inf = DataLoader(dataset_inf, sampler=SequentialSampler(dataset_inf), batch_size=32)

In [66]:
# run batch inference for quantized model
%%timeit
device = "cpu"
all_logits = np.empty([0,2])
# shape is (0, 2), 2 cols for 2 probabilities (positive and negative)

for batch in dataloader_inf:

    batch = tuple(b.to(device) for b in batch)
    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              }
    with torch.no_grad(): # with torch >= 1.9 can use `with torch.inference_mode():` to be even faster         
        outputs = model_dynamic_quantized(**inputs)

    logits = outputs[0]

    all_logits = np.vstack([all_logits, torch.softmax(logits, dim=1).detach().cpu().numpy()])

10 loops, best of 5: 50.8 ms per loop


In [67]:
# batch inference for jit traced quantized model
%%timeit
device = "cpu"
all_logits = np.empty([0,2])
# shape is (0, 2), 2 cols for 2 probabilities (positive and negative)

for batch in dataloader_inf:

    batch = tuple(b.to(device) for b in batch)
    input_ids, input_mask = batch[0], batch[1]

    with torch.no_grad():        
        outputs = traced_q_model(input_ids, input_mask)
    logits = outputs["logits"]
    all_logits = np.vstack([all_logits, torch.softmax(logits, dim=1).detach().cpu().numpy()])

all_logits

10 loops, best of 5: 32.5 ms per loop
